In [6]:
%run '/home/christianl/Zhang-Lab/Zhang Lab Code/Boilerplate_datahandling/Remote boilerplate/model_boilerplate_remote.py'

In [8]:
# out-of-the-box r2 score LinearRegression:
# train score: 0.90870454039286
# test score:  0.7934193600256717 
# difference of 0.12 between train and test is understandable given the high-dimensionality and noise in gene expression data

# after applying the centering boilerplate (only difference between MLR v1 and v2 is column-wise centering)
# train score: 0.90870454039286
# test score:  0.7934193600256717 

from sklearn.linear_model import LinearRegression

reg_test = LinearRegression(
                    n_jobs=-1,
).fit(x_train, y_train)

In [9]:
print('Score: ', reg_test.score(x_train, y_train))
print('Score: ', reg_test.score(x_test, y_test))

Score:  0.9086830465567495
Score:  0.7995409712262469


In [10]:
# conducting a k-fold validation to check for overfitting
# splitting training set into 10-folds
# train sets: ~10038 targets
# test sets: ~1116 targets

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

cnt = 0
n_splits = 10
kf = KFold(n_splits=n_splits, shuffle=True, random_state=888) # changed from 42 to 888 to match training seed for RNN 13/01/26
for train_index, test_index in kf.split(x_train, y_train):
    print(f'Fold:{cnt}, Train set: {len(train_index)}, \
    Test set:{len(test_index)}')
    cnt += 1

Fold:0, Train set: 10038,     Test set:1116
Fold:1, Train set: 10038,     Test set:1116
Fold:2, Train set: 10038,     Test set:1116
Fold:3, Train set: 10038,     Test set:1116
Fold:4, Train set: 10039,     Test set:1115
Fold:5, Train set: 10039,     Test set:1115
Fold:6, Train set: 10039,     Test set:1115
Fold:7, Train set: 10039,     Test set:1115
Fold:8, Train set: 10039,     Test set:1115
Fold:9, Train set: 10039,     Test set:1115


In [ ]:
# running the 10-fold validation 
#----- MLR model cross validation ------
#Scores: [0.78811923 0.79602325 0.7830914  0.78294147 0.78665107 0.7712694
# 0.77717178 0.7840341  0.79817987 0.76680642]
#Mean: 0.7834287993694773 -> similar to the holdout training run
#StandardDeviation: 0.009341412328795035 -> very consistent between folds

# unlikely that MLR is overfitting based on these results -> good sanity check for reference

# redid same analysis with column-wise centered data and got the exact same results 
#----- MLR model cross validation ------
#Scores: [0.78811923 0.79602325 0.7830914  0.78294147 0.78665107 0.7712694
# 0.77717178 0.7840341  0.79817987 0.76680642]
#Mean: 0.7834287993694773
#StandardDeviation: 0.00934141232879502

# redid same analysis once again, with uncentered data like the first run, but set random seed to 888
#----- MLR model cross validation ------
#Scores: [0.79322189 0.79384094 0.80615516 0.77715731 0.78278073 0.78325776
# 0.79921787 0.77486414 0.75888273 0.7807757 ]
#Mean: 0.7850154223167911
#StandardDeviation: 0.01291359135998445


def cross_validation(reg_model, training_set, training_target, cv):
    scores = cross_val_score(
      reg_model, training_set,
      training_target,
      scoring="r2", cv=cv)
    r2_scores = scores
    print("Scores:", r2_scores)
    print("Mean:", r2_scores.mean())
    print("StandardDeviation:", r2_scores.std())

print("----- MLR model cross validation ------")
lin_reg = LinearRegression()
cross_validation(lin_reg, x_train, y_train, kf)
print("")

----- MLR model cross validation ------
Scores: [0.79322189 0.79384094 0.80615516 0.77715731 0.78278073 0.78325776
 0.79921787 0.77486414 0.75888273 0.7807757 ]
Mean: 0.7850154223167911
StandardDeviation: 0.01291359135998445



In [16]:
import joblib
import json

# saving model + metadata

metadata = {
    'sklearn_version': '1.7.2',
    'model_type': 'LinearRegression',
    'n_features': reg_test.n_features_in_,
    'coef_shape': reg_test.coef_.shape,
    'intercept_shape': reg_test.intercept_.shape,
}

joblib.dump(reg_test, "/home/christianl/Zhang-Lab/Zhang Lab Data/Saved models/MLR/MLR_v3/MLR_model_v4(uncentered[FINAL]).joblib")
with open("/home/christianl/Zhang-Lab/Zhang Lab Data/Saved models/MLR/MLR_v3/model_metadata_uncentered[FINAL].json", "w") as f:
    json.dump(metadata, f)